<a href="https://colab.research.google.com/github/hammad93/hurricane-tts/blob/main/hurricane_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation
Please make sure to run all tests.

In [1]:
%cd ~/
!git clone https://github.com/hammad93/hurricane-tts.git

/root
Cloning into 'hurricane-tts'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 137 (delta 76), reused 73 (delta 29), pack-reused 0
Receiving objects: 100% (137/137), 1.16 MiB | 7.06 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [2]:
!pip install -r hurricane-tts/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
import os
if not os.getenv("AZURE_OPENAI_API_KEY") :
  from google.colab import userdata
  os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
if not os.getenv("AZURE_REDIS_KEY") :
  os.environ["AZURE_REDIS_KEY"] = userdata.get('AZURE_REDIS_KEY')

In [4]:
%cd ~/hurricane-tts
!python ~/hurricane-tts/test.py

/root/hurricane-tts
.Here are the constructed messages: [{'role': 'system', 'content': 'You are an AI assistant that helps people find information.'}, {'role': 'user', 'content': 'test'}]
..
----------------------------------------------------------------------
Ran 3 tests in 7.804s

OK


In [5]:
import prompts
import utils
import db
# we generate prompts by ingesting live hurricane data and supported languages
storm_data = utils.transform_storm_data()
prompt_data = prompts.generate_prompts()

# metadata related to multilingual generation
supported_langs = prompts.unique_lang_list()
supported_langs_df = utils.tts_langs()

In [6]:
# example
print(prompt_data['storms'][0])

These are the input storm records and forecasts.
Each record has a lat and lon according to their geographic coordinates and wind speed in knots.
Please respond with the 3 most relevant languages exactly as they appear from the supported language list other than English each delimited by a comma.
[{'type': 'history', 'lat': 5.1, 'lon': 149.0, 'time': '2023-11-09 18:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 5.3, 'lon': 148.4, 'time': '2023-11-10 00:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 5.5, 'lon': 147.8, 'time': '2023-11-10 06:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 5.7, 'lon': 147.5, 'time': '2023-11-10 12:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 6.0, 'lon': 147.2, 'time': '2023-11-10 18:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 6.1, 'lon': 146.7, 'time': '2023-11-11 00:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 6.1, 'lon': 146.0, 'time': '2023-11-11 06:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 6.

# Language Geography inference from tropical storm geographical coorindates
We're trying to answer the question, "Which languages do we produce speech for to report on this tropical storm?" This pipeline will produce an English output but also utilize massively multilingual capabilities such that emergency notices are also in local languages

In [7]:
retries = 5 # sometimes it fails, so we retry it
storm_langs = {} # keys are storm id's and values are the languages
storm_chats = {} # stores chat histories for storms
for index, storm_prompt in enumerate(prompt_data['storms']) :
  storm_id = list(storm_data.keys())[index]
  # Get the languages to generate the report for this storm
  while retries > 0 :
    response = prompts.chat(system=prompt_data['system'], message=storm_prompt)
    print(response)
    result = utils.llm_response_transform(
        resp=response, supported_langs=supported_langs)
    print(result)
    if result :
      break
    else :
      retries = retries - 1
      print(f"Failed. Retries left: {retries}")
  if retries < 1 :
    raise Exception("Couldn't produce a correct output from LLM.")

  # store results
  storm_langs[storm_id] = {'names': ['English'] + result}
  storm_chats[storm_id] = {
      'history' : [{'role': 'system', 'content': prompt_data['system']},
                   {'role': 'user', 'content': storm_prompt},
                   {'role': 'assistant', 'content': response}]
  }

Here are the constructed messages: [{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nI have a system for emergency notification of tropical storms that utilizes official data sources and creates speech audio from text with a massively multilingual model.\nThis is the list of supported languages,\n{'Mongolian', 'NambikuÃ¡ra, Southern', 'Tohono Oâ\\x80\\x99odham', 'Tii', 'Buglere', 'Cuiba', 'Tibetan, Amdo', 'Gujarati', 'Suba', 'Kumyk', 'Aymara, Central', 'Lole', 'Maranao', 'Ndogo', 'Nanai', 'Adioukrou', 'Kandozi-Chapra', 'Indonesian', 'Ikwo', 'Pangasinan', 'Hdi', 'Mayo', 'Quechua, Ayacucho', 'Nyakyusa-Ngonde', 'Makaa', 'Fulah', 'Assamese', 'Garifuna', 'Pitjantjatjara', 'Chiduruma', 'Samburu', 'Lamnsoâ\\x80\\x99', 'Bughotu', 'Quechua, Cusco', 'Juray', 'GourmanchÃ©ma', 'Benga', 'Tehit', 'PÃ©vÃ©', 'Bantoanon', 'Bulgarian', 'Meyah', 'Napu', 'Kukele', 'Chinantec, Lalana', 'Mixe, Juquila', 'Toura', 'Vagla', 'Wolaytta', 'Bomu', 'Enxet', 'S

# Tropical Storm Report
This code will create the report in the language specified based on the tropical storm.

In [8]:
with open('prompts/report-prompt.txt', 'r') as file:
  report_prompt = file.read()
for storm in storm_langs:
  reports = []
  for lang in storm_langs[storm]['names']:
    print(storm_chats[storm]['history'])
    # Construct the prompt
    message = report_prompt.format(lang=lang)
    print(message)
    # Send to LLM
    response = prompts.chat(
        message = message, history = storm_chats[storm]['history'])
    print(response)
    # store data
    reports.append(response)
  storm_langs[storm]['reports'] = reports

[{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nI have a system for emergency notification of tropical storms that utilizes official data sources and creates speech audio from text with a massively multilingual model.\nThis is the list of supported languages,\n{'Mongolian', 'NambikuÃ¡ra, Southern', 'Tohono Oâ\\x80\\x99odham', 'Tii', 'Buglere', 'Cuiba', 'Tibetan, Amdo', 'Gujarati', 'Suba', 'Kumyk', 'Aymara, Central', 'Lole', 'Maranao', 'Ndogo', 'Nanai', 'Adioukrou', 'Kandozi-Chapra', 'Indonesian', 'Ikwo', 'Pangasinan', 'Hdi', 'Mayo', 'Quechua, Ayacucho', 'Nyakyusa-Ngonde', 'Makaa', 'Fulah', 'Assamese', 'Garifuna', 'Pitjantjatjara', 'Chiduruma', 'Samburu', 'Lamnsoâ\\x80\\x99', 'Bughotu', 'Quechua, Cusco', 'Juray', 'GourmanchÃ©ma', 'Benga', 'Tehit', 'PÃ©vÃ©', 'Bantoanon', 'Bulgarian', 'Meyah', 'Napu', 'Kukele', 'Chinantec, Lalana', 'Mixe, Juquila', 'Toura', 'Vagla', 'Wolaytta', 'Bomu', 'Enxet', 'Somali', 'Ngangam', 'Mango', 'Waray-

In [9]:
storm_langs

{'WP172023': {'names': ['English', 'Indonesian', 'Tagalog', 'Malay'],
  'reports': ["Attention all residents! Tropical Storm Activity.\nThe tropical storm is moving closer to our location. As of the latest report from official sources, the storm is currently located at latitude 11.1 and longitude 133.0 and has a wind speed of 50 knots. Updates show that the storm's path is changing, and it is expected to hit our area tomorrow evening with wind speeds of up to 45 knots. Please take precautions and prepare for strong winds and heavy rains. Stay indoors, and avoid any unnecessary travel. Keep a close eye on official updates and listen for emergency alerts. Stay safe and be prepared.",
   'Peringatan Badai Tropis: Terdapat badai tropis di sekitar koordinat 5.1 LS dan 149.0 BT yang bergerak ke arah barat daya. Diperkirakan badai tropis akan terus bergerak ke arah barat daya dan mencapai koordinat 11.1 LS dan 133.0 BT pada tanggal 15 November 2023 pukul 18:00 dengan kecepatan angin mencapai 

The following are from the TTS implementation
https://github.com/facebookresearch/fairseq/blob/main/examples/mms/tts/tutorial/MMS_TTS_Inference_Colab.ipynb


In [10]:
%cd ~/
!git clone https://github.com/jaywalnut310/vits.git
%cd vits/

!pip install Cython==0.29.21
!pip install librosa==0.8.0
!pip install phonemizer==2.2.1
!pip install scipy
!pip install "numpy<1.24"
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install Unidecode==1.1.1

%cd monotonic_align/
%mkdir monotonic_align
!python3 setup.py build_ext --inplace
%cd ../
%pwd

/root
Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81
Receiving objects: 100% (81/81), 3.33 MiB | 19.07 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/root/vits
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.2/974.2 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.5
    Uninstalling Cython-3.0.5:
      Successfully uninstalled Cython-3.0.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.2 MB/s eta 0:00:00
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201367 sha256=b7585e57c0990aea6538ed931e246c64d55291b2c644ad212c1a6c39b410c31b
  Stored in directory: /root/.cache/pip/wheels/bf/b7/85/2f8044306ccec014930aea23ad4852fca9e2584e21c6972bc6
Successfully built librosa
  Attempting uninstall: librosa
  

'/root/vits'

In [11]:
# we have the data we need and the utils class is also used in the next code
import sys
del sys.modules["utils"]

In [12]:
%cd ~/hurricane-tts/
import fair_mms_tts
%cd ~/vits
import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn

/root/hurricane-tts
/root/vits


In [13]:
# match language names to iso code
for storm in storm_langs :
  langs = storm_langs[storm]['names']
  print(langs)
  codes = [supported_langs_df[
      supported_langs_df['Language Name'] == name
  ]['ISO 693-3'].iloc[0] for name in langs]
  print(codes)
  storm_langs[storm]['codes'] = codes

['English', 'Indonesian', 'Tagalog', 'Malay']
['eng', 'ind', 'tgl', 'zlm']
['English', 'Indonesian', 'Samoan', 'Fijian']
['eng', 'ind', 'smo', 'fij']
['English', 'Indonesian', 'Tagalog', 'Malagasy']
['eng', 'ind', 'tgl', 'mlg']
['English', 'Indonesian', 'Tagalog', 'Thai']
['eng', 'ind', 'tgl', 'tha']


In [14]:
import soundfile as sf
def generate_audio(report, lang_code, out):
  '''
  Generates the audio given the report and lang_code
  '''
  ckpt_dir = fair_mms_tts.download(lang_code)
  import torch
  if torch.cuda.is_available():
      device = torch.device("cuda")
  else:
      device = torch.device("cpu")
  print(device)
  print(f"Run inference with {device}")
  vocab_file = f"{ckpt_dir}/vocab.txt"
  config_file = f"{ckpt_dir}/config.json"
  assert os.path.isfile(config_file), f"{config_file} doesn't exist"
  hps = utils.get_hparams_from_file(config_file)

  text_mapper = fair_mms_tts.TextMapper(vocab_file)
  net_g = SynthesizerTrn(
      len(text_mapper.symbols),
      hps.data.filter_length // 2 + 1,
      hps.train.segment_size // hps.data.hop_length,
      **hps.model)
  net_g.to(device)
  _ = net_g.eval()

  g_pth = f"{ckpt_dir}/G_100000.pth"
  print(f"load {g_pth}")

  _ = utils.load_checkpoint(g_pth, net_g, None)
  txt = fair_mms_tts.preprocess_text(report, text_mapper, hps, lang=lang_code)
  stn_tst = text_mapper.get_text(txt, hps)

  with torch.no_grad():
      x_tst = stn_tst.unsqueeze(0).to(device)
      x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
      hyp = net_g.infer(
          x_tst, x_tst_lengths, noise_scale=.667,
          noise_scale_w=0.8, length_scale=1.0
      )[0][0,0].cpu().float().numpy()

  print(f"Generated audio")
  sf.write(out, hyp, hps.data.sampling_rate)

In [15]:
# generate audio
import time
time_fname = int(time.time())
outputs = []
for storm in storm_langs :
  current = storm_langs[storm]
  for index, code in enumerate(current['codes']):
    path = f"~/{storm}_{time_fname}_{current['names'][index]}.wav"
    outputs.append(path)
    generate_audio(report = current['reports'][index],
                   lang_code = code,
                   out = path)

Download model for language: eng
Model checkpoints in ./eng: ['vocab.txt', 'config.json', 'G_100000.pth']
cuda
Run inference with cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


load ./eng/G_100000.pth
eng
text after filtering OOV: attention all residents tropical storm activitythe tropical storm is moving closer to our location as of the latest report from official sources the storm is currently located at latitude 111 and longitude 1330 and has a wind speed of 50 knots updates show that the storm's path is changing and it is expected to hit our area tomorrow evening with wind speeds of up to 45 knots please take precautions and prepare for strong winds and heavy rains stay indoors and avoid any unnecessary travel keep a close eye on official updates and listen for emergency alerts stay safe and be prepared
Generated audio
Download model for language: ind
Model checkpoints in ./ind: ['vocab.txt', 'config.json', 'G_100000.pth']
cuda
Run inference with cuda
load ./ind/G_100000.pth
ind
text after filtering OOV: peringatan badai tropis terdapat badai tropis di sekitar koordinat 51 ls dan 140 bt yang bergerak ke arah barat daya diperkirakan badai tropis akan terus

In [16]:
import db
import json
print(outputs)
db.upload_latest_audios(outputs)

['/content/WP172023_1699834654_English.wav', '/content/WP172023_1699834654_Indonesian.wav', '/content/WP172023_1699834654_Tagalog.wav', '/content/WP172023_1699834654_Malay.wav', '/content/SH912023_1699834654_English.wav', '/content/SH912023_1699834654_Indonesian.wav', '/content/SH912023_1699834654_Samoan.wav', '/content/SH912023_1699834654_Fijian.wav', '/content/WP962023_1699834654_English.wav', '/content/WP962023_1699834654_Indonesian.wav', '/content/WP962023_1699834654_Tagalog.wav', '/content/WP962023_1699834654_Malagasy.wav', '/content/WP952023_1699834654_English.wav', '/content/WP952023_1699834654_Indonesian.wav', '/content/WP952023_1699834654_Tagalog.wav', '/content/WP952023_1699834654_Thai.wav']
